In [50]:
# Imports

import os
import requests
import json
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
import mistune
from langchain import OpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.tools import BaseTool

In [51]:
# Setup

load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

if not OPENROUTER_API_KEY:
    raise ValueError("Please set the OPENROUTER_API_KEY environment variable.")

API_URL = "https://openrouter.ai/api/v1/chat/completions"

HEADERS = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json",
}

In [52]:
# API Functions

def call_openrouter(message, model):

    payload = {
        "model": model,
        "messages": [
            {"role": "user", "content": message}
        ]
    }
    
    response = requests.post(API_URL, headers=HEADERS, data=json.dumps(payload))

    if response.status_code == 200:
        data = response.json()
        if "choices" not in data:   
            raise Exception(f"Unexpected response format: {data}")
        return data["choices"][0]["message"]["content"]

        return data["choices"][0]["message"]["content"]
    else:
        raise Exception(f"Request failed with status {response.status_code}: {response.text}")



def agent(prompt, model):

    print("Agent starting...")
    print("Sending prompt:", prompt)
    
    try:
        response_text = call_openrouter(prompt, model)
        display(Markdown(response_text))

        markdown = mistune.create_markdown(renderer='ast')
        tokens = markdown(response_text)
        # print(tokens)
                
    except Exception as e:
        print("\nError during API call:\n", e)


In [53]:
# Two agent conversation 

def conversation_session(agent1_model, agent2_model, initial_message, num_turns=5):
   
    conversation_history = []
    current_message = initial_message
    
    print("Starting conversation...\n")
    print("Agent1:", current_message)

    conversation_history.append(("Agent1", current_message))
    

    for turn in range(num_turns):
        prompt_for_agent2 = "The conversation so far:\n"
        for speaker, msg in conversation_history:
            prompt_for_agent2 += f"{speaker}: {msg}\n"

        prompt_for_agent2 += "\nAgent1 just spoke. Agent2, please respond."
        try:
            agent2_response = call_openrouter(prompt_for_agent2, agent2_model)
        except Exception as e:
            print(f"Error from Agent2: {e}")
            agent2_response = "[Error in Agent2 response]"

        print("Agent2:", agent2_response)
        conversation_history.append(("Agent2", agent2_response))
        
        prompt_for_agent1 = "The conversation so far:\n"
        for speaker, msg in conversation_history:
            prompt_for_agent1 += f"{speaker}: {msg}\n"

        prompt_for_agent1 += "\nAgent2 just spoke. Agent1, please respond."
        try:
            agent1_response = call_openrouter(prompt_for_agent1, agent1_model)
        except Exception as e:
            print(f"Error from Agent1: {e}")
            agent1_response = "[Error in Agent1 response]"

        print("Agent1:", agent1_response)
        conversation_history.append(("Agent1", agent1_response))
    
    return conversation_history


In [ ]:
# Define a tool to send emails (using a placeholder for actual email API calls)
class EmailTool(BaseTool):
    name = "send_email"
    description = "Sends an email with a subject and body to a given recipient."

    def _run(self, subject: str, body: str, recipient: str) -> str:
        # Replace the following line with actual code that integrates with your email API
        return f"Email sent to {recipient} with subject '{subject}' and body '{body[:50]}...'."

    async def _arun(self, subject: str, body: str, recipient: str) -> str:
        raise NotImplementedError("Async not implemented")


# Define a tool to scrape web content
class WebScraperTool(BaseTool):
    name = "scrape_web"
    description = "Scrapes and returns plain text from the given URL."

    def _run(self, url: str) -> str:
        import requests
        from bs4 import BeautifulSoup
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.get_text(separator="\n")  # Return the text with line breaks

    async def _arun(self, url: str) -> str:
        raise NotImplementedError("Async not implemented")


# Define a tool to extract YouTube video details (placeholder)
class YouTubeTool(BaseTool):
    name = "scrape_youtube"
    description = "Retrieves information about a YouTube video given its URL."

    def _run(self, url: str) -> str:
        # In production, you might call the YouTube Data API or use a library like youtube_dl.
        return f"Details for YouTube video at {url} (dummy data)."

    async def _arun(self, url: str) -> str:
        raise NotImplementedError("Async not implemented")


# Define a tool to generate a report from data
class ReportGeneratorTool(BaseTool):
    name = "generate_report"
    description = "Generates a summary report from the provided data string."

    def _run(self, data: str) -> str:
        # This is a simplified example; you could use a templating engine or even ask the LLM to produce a report.
        return f"Report Summary: {data[:100]}..."  # Dummy summary taking the first 100 characters

    async def _arun(self, data: str) -> str:
        raise NotImplementedError("Async not implemented")




llm = OpenAI(temperature=0)

# Register our tools with the agent
tools = [EmailTool(), WebScraperTool(), YouTubeTool(), ReportGeneratorTool()]

# Create the agent using LangChain’s helper function
agent = create_openai_tools_agent(llm, tools, verbose=True)

# Wrap it in an AgentExecutor for easy invocation
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Example usage: the agent will decide which tool to call based on the query
# For instance, a query that requires scraping and sending a report:
query = (
    "Send an email with subject 'Daily Report' to [email protected] "
    "that summarizes the latest news from https://cnn.com."
)

result = agent_executor.invoke({"input": query})
print(result)

In [ ]:
# Config 

gemini_2_flash_lite = 'google/gemini-2.0-flash-lite-preview-02-05:free'
gemini_2_pro_exp = 'google/gemini-2.0-pro-exp-02-05:free'
gemini_2_flash_thinking_exp = 'google/gemini-2.0-flash-thinking-exp:free'

prompt = 'Discuss building a short term trading strategy for BTCUSDT. Scalping is the goal on the 1-1hr time frames. The strategy should be based on the price and volume data not all these other indicators.\n'

# response = agent(prompt, gemini_2_flash_lite)

history = conversation_session(gemini_2_pro_exp, gemini_2_pro_exp, prompt, num_turns=50)